# for loop vs for-each
---

## Performance

- Timing the difference between iterator and index access for primitive int-arrays: Indexes are 23-40 percent faster than iterators when accessing int or Integer arrays.

- For collections, iterators are faster than indexes. 

https://stackoverflow.com/a/22540191/16317008

## 两个循环如何选择?


- 删除就用iterator, **且**得用iterator的`remove()`方法, 否则会导致Iterator Invalidation问题. 

- 添加元素的时候用for loop, 因为`size()`方法会实时更新collection的大小

- for-each遍历array的时候比较慢相对index access

- 删除的时候不能用for-each(抛出异常)和for loop(会出现元素被跳过问题)

删除的时候用for-each会报错 e.g.,
```java
public class Main {
    public static void main(String[] args) {
        List<Integer> list = new ArrayList<>();
        list.add(1);
        list.add(2);

        for (int item : list) {
            if (item == 2) {
                list.remove(item);
            }
        }

        System.out.println(list);
    }
}

// Exception in thread "main" java.lang.IndexOutOfBoundsException ...
```

删除的时候用for loop会跳过元素 e.g.,
```java
import java.util.ArrayList;

public class Main {
    public static void main(String[] args) {
        ArrayList<Integer> list = new ArrayList<>();
        list.add(1);
        list.add(2);
        list.add(3);

        for (int i = 0; i < list.size(); ++i){
            System.out.println(list.get(i));
            if (list.get(i) == 2) {
                list.remove(i);
            }
        }
    }
}

// 1
// 2
```

因为删除一个元素后, size变成了2, 此时i已经是1了, ++i之后, 就会直接跳出循环, 而不会打印数字3. 


> A foreach is equivalent to an iterator--it's **syntactic sugar** for the same thing. So **you should always choose foreach over iterator** whenever you can, simply because it's convenient and results in more concise code. https://stackoverflow.com/a/22540191/16317008

# Iterator
---

```java
ArrayList<String> names = new ArrayList<>();
names.add("123");
names.add("1234");
Iterator<String> iterator = names.iterator();
```

`Iterator<String>`是interface, **在源码`Iterator.java`中**, 定义如下:

```java
public Iterator<E> iterator() {
        return new Itr();
}
```

`names.iterator()`返回一个`Itr` object, **在源码`ArrayList.java`中**, 可以看到有`next()`, hasNext()方法, 而当前位置也就是下标存储在Itr对象的cursor中, 定义如下:
```java
private class Itr implements Iterator<E> {
        int cursor;       // index of next element to return
        int lastRet = -1; // index of last element returned; -1 if no such
        int expectedModCount = modCount;

        // prevent creating a synthetic constructor
        Itr() {}

        public boolean hasNext() {
            return cursor != size;
        }

        @SuppressWarnings("unchecked")
        public E next() {
        ...
        ...
        ...
```


看到这, 相信谁是Interface, 谁是class已经很清晰了. 

#  Iterator的三个method
---

迭代器是个接口, 主要规定了三个函数`hasNext()`, `next()` 和 `remove()`. 

### 1) `hasNext()`

`hasNext()` 只是判断是不是遍历到头了, 并未对下标进行修改. 

```java
public boolean hasNext() {
    return cursor != size;
}
```

### 2) `next()` 

`next()`返回的是**当前下标cursor所指的对象**, 然后`cursor++`(具体看源码, 其实返回是`lastRet`)

### 3) `remove()`

这个最容易犯错, 看了迭代器的源码会发现里面有连两个fields, `cursor`和`lastRet`, 也就是说`next()`和`remove()`每次都是操作的`lastRet`.

**调用`remove()`前应该先调用`next()`方法**让`lastRet`加一, 否则两种情况:

1. 你从来没用过`next()`, 然后`lastRet`的值是-1, 所以会抛出一场
2. 你之前用过`next()`, 但本次循环未用, 那删除的则是上次循环的元素. 

# Iterator Invalidation
----

- 创建迭代器后不能再通过**容器的`add/remove`方法**来改变容器的数据，否则会导致迭代器的失效

```java
import java.util.ArrayList;
import java.util.Iterator;

public class Main {
    public static void main(String[] args) {
        ArrayList<Integer> list = new ArrayList<>();
        list.add(1);
        list.add(2);

        Iterator<Integer> iterator = list.iterator();
        while (iterator.hasNext()) {
            Integer element = iterator.next();
            System.out.println(element);
            list.add(3); // This will invalidate the iterator
        }
    }
}
```

### 避免方法

- 使用for loop对collection进行增删, 没事别用iterator, 注意for loop不等于for each, 差别老大了

- 用iterator提供的remove方法去remove元素

iterator并未提供add方法, 为什么不提供呢, 因为有可能产生无限循环infinite loop: https://stackoverflow.com/a/68540984/16317008

但是想用iterator然后还想添加的时候怎么办呢? 答: 别没事找事 (狗头


# for-each
---

`for-each`本质上是个**syntactic sugar**，底层是通过`Iterator`配合`while`循环实现的, 

```java
List<String> list = new ArrayList<>();
list.add("沉默王二");
list.add("沉默王三");
list.add("一个文章真特么有趣的程序员");

for (String str : list) {
	if ("沉默王二".equals(str)) {
		list.remove(str);
	}
}

System.out.println(list);
```

来看一下反编译后的字节码:

```java
List<String> list = new ArrayList();
list.add("沉默王二");
list.add("沉默王三");
list.add("一个文章真特么有趣的程序员");
Iterator var2 = list.iterator();

while(var2.hasNext()) {
    String str = (String)var2.next();
    if ("沉默王二".equals(str)) {
        list.remove(str);
    }
}

System.out.println(list);
```